In [ ]:
import cv2
from tqdm import tqdm_notebook as tqdm
import fastai
from fastai.vision import *
import os
from mish_activation import *
import warnings
warnings.filterwarnings("ignore")
import skimage.io
import numpy as np
import pandas as pd
sys.path.insert(0, '../input/semisupervised-imagenet-models/semi-supervised-ImageNet1K-models-master/')
from hubconf import *
from bisect import bisect_right

In [ ]:
#DATA = '../input/prostate-cancer-grade-assessment/train_images/'
#TEST = '../input/prostate-cancer-grade-assessment/train.csv'
DATA = '../input/prostate-cancer-grade-assessment/test_images'
TEST = '../input/prostate-cancer-grade-assessment/test.csv'
SAMPLE = '../input/prostate-cancer-grade-assessment/sample_submission.csv'
#MODELS = [f'../input/panda-init-class-model1/RNXT50_128kr_2_{i}.pth' for i in range(4)]
MODELS = [f'../input/panda-init-class-model1/RNXT50_128k_0_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128k_3_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_1_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_2_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_2feature_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_9_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_7c_{i}.pth' for i in range(4)] + \
         [f'../input/panda-init-class-model1/RNXT50_128kr_3_{i}.pth' for i in range(4)]
ws = [1.0,1.0,1.0,7.0,1.0,1.0,1.0,1.0]
#MODELS = [f'../input/panda-init-class-model1/RNXT50_128kr_6feature_{i}.pth' for i in range(4)] + \
#         [f'../input/panda-init-class-model1/RNXT50_128kr_2feature_{i}.pth' for i in range(4)]
#MODELS = [f'../input/panda-init-class-model1/RNXT50_256k_1_{i}.pth' for i in range(4)] + \
#         [f'../input/panda-init-class-model1/RNXT50_256c1_0_{i}.pth' for i in range(4)] #+ \
         #[f'../input/panda-init-class-model1/RNXT50_256_0_{i}.pth' for i in range(4)]

sz = 128
bs = 1
N = 128
nworkers = 2

In [ ]:
def _resnext(url, block, layers, pretrained, progress, **kwargs):
    model = ResNet(block, layers, **kwargs)
    #state_dict = load_state_dict_from_url(url, progress=progress)
    #model.load_state_dict(state_dict)
    return model

class Model(nn.Module):
    def __init__(self, arch='resnext50_32x4d', n=6, pre=True):
        super().__init__()
        #m = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', arch)
        m = _resnext(semi_supervised_model_urls[arch], Bottleneck, [3, 4, 6, 3], False, progress=False,\
                     groups=32,width_per_group=4)
        self.enc = nn.Sequential(*list(m.children())[:-2])       
        nc = list(m.children())[-1].in_features
        #self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),nn.Linear(2*nc,512),Mish(),nn.BatchNorm1d(512),
        #                          nn.Dropout(0.5),nn.Linear(512,n))
        self.head = nn.Sequential(AdaptiveConcatPool2d(),Flatten(),
                                  nn.Linear(2*nc,512),Mish(),nn.GroupNorm(32,512),
                                  nn.Dropout(0.5),nn.Linear(512,n))
        #self.head_s = nn.Conv2d(nc,7,1)
        
    def forward(self, x):
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        x = self.enc(x)
        shape = x.shape
        x = x.view(-1,n,shape[1],shape[2],shape[3]).permute(0,2,1,3,4).contiguous()\
          .view(-1,shape[1],shape[2]*n,shape[3])
        x = self.head(x)
        return x[:,:1]

In [ ]:
models = []
for path in MODELS:
    state_dict = torch.load(path,map_location=torch.device('cpu'))
    model = Model(n=1+10)
    #model = Model()
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()
    models.append(model)

del state_dict

In [ ]:
def tile(img):
    shape = img.shape
    pad0,pad1 = (sz - shape[0]%sz)%sz, (sz - shape[1]%sz)%sz
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],constant_values=255)
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)
    if len(img) < N:
        img = np.pad(img,[[0,N-len(img)],[0,0],[0,0],[0,0]],constant_values=255)
    idxs = np.argsort(img.reshape(img.shape[0],-1).sum(-1))[:N]
    img = img[idxs]
    return img

mean = torch.Tensor([1.0-0.85506157, 1.0-0.7035249, 1.0-0.80203127])
std = torch.Tensor([0.40011922, 0.52504386, 0.42675745])

class PandaDataset(Dataset):
    def __init__(self, path, test):
        self.path = path
        self.names = list(pd.read_csv(test).image_id)

    def __len__(self):
        return len(self.names)

    def __getitem__(self, idx):
        name = self.names[idx]
        img = skimage.io.MultiImage(os.path.join(DATA,name+'.tiff'))[1]
        tiles = torch.Tensor((255 - tile(img))/255.0)
        tiles = (tiles - mean)/std
        return tiles.permute(0,3,1,2), name

In [ ]:
ths = np.array([1.03125,1.03125,0.8125,0.90625,1.0859375]).cumsum()
#ths = np.array([1.0,1.0,1.0,1.0,1.0]).cumsum()
#ths = np.array([1.11,2.10,3.13,3.92,4.93])
#ths = np.array([1.06,1.92,3.05,3.88,4.91])
#ths = np.array([1.03,2.11,2.90,3.81,4.88])

ws = torch.Tensor(ws).cuda()
sub_df = pd.read_csv(SAMPLE)
if os.path.exists(DATA):
    bs=2
    ds = PandaDataset(DATA,TEST)
    dl = DataLoader(ds, batch_size=bs, num_workers=nworkers, shuffle=False)
    names,preds = [],[]

    with torch.no_grad():
        for x,y in tqdm(dl):
            x = x.cuda()
            b = x.shape[0]
            #dihedral TTA
            #x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),x.transpose(-1,-2),\
            #  x.transpose(-1,-2).flip(-1), x.transpose(-1,-2).flip(-2),\
            #  x.transpose(-1,-2).flip(-1,-2)],1)
            x = torch.stack([x,x.flip(-1),x.flip(-2),x.flip(-1,-2),x.transpose(-1,-2),\
              x.transpose(-1,-2).flip(-1)],1)
            n_tta = 6
            x = x.view(-1,N,3,sz,sz)
            #p = [model(x[a].unsqueeze(0)) for model in models[:8] for a in range(x.shape[0])]
            p = [model(x) for model in models]
            p = torch.stack(p,1)
            p = p.view(b,n_tta,len(models)//4,4).permute(0,2,3,1).reshape(b,len(models)//4,4*n_tta)
            w = ws/torch.clamp(p.std(-1),0.04)
            w /= w.sum(-1).unsqueeze(-1)
            #print(w)
            #print(6.0*torch.sigmoid((p)))
            #print(w,6.0*torch.sigmoid((p*w.unsqueeze(-1)).mean(-1).sum(-1).cpu()),
            #      6.0*torch.sigmoid(p.mean(-1).mean(-1).cpu()))
            p = (p*w.unsqueeze(-1)).mean(-1).sum(-1).cpu()
            p = 6.0*torch.sigmoid(p).view(-1)
            #print(p)
            for pi,yi in zip(p,y):
                preds.append(bisect_right(ths, pi.numpy()))
                names.append(yi)
            
            #p = (p*w.unsqueeze(-1)).mean(-1).sum(-1).cpu()
            #p = torch.clamp((6.0*torch.sigmoid(p)).long().view(-1),0,5)
            ###p = p.view(bs,8*len(models),-1).mean(1).argmax(-1).cpu()
            ##p = p.view(b,n_tta*len(models)).mean(1).cpu()
            ##p = torch.clamp((6.0*torch.sigmoid(p)).long().view(-1),0,5)
            #names.append(y)
            #preds.append(p)
    
    #names = np.concatenate(names)
    #preds = torch.cat(preds).numpy()
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': preds})

In [ ]:
sub_df.to_csv("submission.csv", index=False)
sub_df.head()